In [1]:
import pandas as pd
import numpy as np

from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout
from keras.layers.wrappers import TimeDistributed
from keras.utils import np_utils
from keras import optimizers

from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import confusion_matrix
import random

Using TensorFlow backend.


In [12]:
train = pd.read_csv('/Users/xiaoyudai/Documents/Paper3/Data/Brain_ES_chr18/crf/train_rnn', 
                    sep='\t', header=None).values
test = pd.read_csv('/Users/xiaoyudai/Documents/Paper3/Data/Brain_ES_chr18/crf/test_rnn', 
                    sep='\t', header=None).values

In [13]:
train.shape

(64716, 34)

In [5]:
train[1,:]

array([  0.00000000e+00,   9.67598082e-07,   0.00000000e+00,
         0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
         0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
         0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
         1.00000000e+00,   1.79505664e-01,   5.00004072e-01,
         1.06759641e-01,   1.74630939e-01,   5.00051625e-01,
         1.62387923e-01,   1.67222786e-01,   5.09866442e-01,
         2.46431705e-01,   1.63501253e-01,   4.91725768e-01,
         7.45216956e-01,   1.66694793e-01,   5.05672620e-01,
         6.98979592e-01,   1.30869904e-02,   3.51405623e-02,
         8.72817955e-03,   7.94912554e-03,   4.13604683e-06,
         1.00000000e+00])

In [14]:
x_train = train[:,0:-1].astype(np.float)
y_train = train[:,-1].astype(np.float)

x_test = test[:,0:-1].astype(np.float)
y_test = test[:,-1].astype(np.float)

y_train = np_utils.to_categorical(y_train)
y_test = np_utils.to_categorical(y_test)

In [15]:
sub_length = 2
x_train = x_train.reshape(x_train.shape[0]/sub_length, sub_length, 33, order='C')
x_test = x_test.reshape(x_test.shape[0]/sub_length, sub_length, 33, order='C')

y_train = y_train.reshape(y_train.shape[0]/sub_length, sub_length, 2)
y_test = y_test.reshape(y_test.shape[0]/sub_length, sub_length, 2)

In [143]:
from keras.preprocessing import sequence

In [149]:
x_train = sequence.pad_sequences(x_train, maxlen=10)
x_test = sequence.pad_sequences(x_test, maxlen=10)

y_train = sequence.pad_sequences(y_train, maxlen=10)
y_test = sequence.pad_sequences(y_test, maxlen=10)

In [145]:
a.shape

(32358, 10, 33)

In [165]:
x_train.shape

(64716, 1, 33)

In [98]:
def normaling_feature(x_train, x_test):
    dt = np.concatenate((x_train, x_test), axis=0)
    scaler = MinMaxScaler(feature_range=(0,1))
    scaler = scaler.fit(dt)
    scaled_dt = scaler.transform(dt)
    return scaled_dt[:x_train.shape[0], :], scaled_dt[x_train.shape[0]:, :]

x_train = train[:,0:-2].astype(np.float)
y_train = train[:,-2].astype(np.float)
y_train = np_utils.to_categorical(y_train)
# y_train = train[:,-1].astype(np.float)

x_test = test[:,0:-2].astype(np.float)
y_test = test[:,-2].astype(np.float)
y_test = np_utils.to_categorical(y_test)
# y_test = test[:,-1].astype(np.float)

scaled_x_train, scaled_x_test = normaling_feature(x_train, x_test)

In [99]:
num_train = scaled_x_train.shape[0]
seq_length = 10

scaled_x_train = scaled_x_train[:num_train/seq_length * seq_length]
scaled_x_train = scaled_x_train.reshape(num_train/seq_length, seq_length, 47, order='C')

y_train = y_train[:num_train/seq_length * seq_length]
y_train = y_train.reshape(num_train/seq_length, seq_length, 2, order='C')

In [100]:
num_test = scaled_x_test.shape[0]

scaled_x_test = scaled_x_test[:num_test/seq_length * seq_length]
scaled_x_test = scaled_x_test.reshape(num_test/seq_length, seq_length, 47, order='C')

y_test = y_test[:num_test/seq_length * seq_length]
y_test = y_test.reshape(num_test/seq_length, seq_length, 2, order='C')

In [16]:
model = Sequential()
model.add(LSTM(256, input_shape=(sub_length, 33), return_sequences=True))
# model.add(LSTM(256, return_sequences=True))
model.add(LSTM(128, return_sequences=True))
model.add(LSTM(32, return_sequences=True))
model.add(TimeDistributed(Dense(2, activation='softmax')))

model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
lstm_1 (LSTM)                    (None, 2, 256)        296960      lstm_input_1[0][0]               
____________________________________________________________________________________________________
lstm_2 (LSTM)                    (None, 2, 128)        197120      lstm_1[0][0]                     
____________________________________________________________________________________________________
lstm_3 (LSTM)                    (None, 2, 32)         20608       lstm_2[0][0]                     
____________________________________________________________________________________________________
timedistributed_1 (TimeDistribut (None, 2, 2)          66          lstm_3[0][0]                     
Total params: 514,754
Trainable params: 514,754
Non-trainable params: 0
___________________

/Users/xiaoyudai/anaconda/lib/python2.7/site-packages/keras/engine/topology.py:368: UserWarning: The `regularizers` property of layers/models is deprecated. Regularization losses are now managed via the `losses` layer/model property.
  warnings.warn('The `regularizers` property of '


In [17]:
stateful_model = Sequential()
stateful_model.add(LSTM(256, batch_input_shape=(1, sub_length, 33), return_sequences=True, stateful=True))
# model.add(LSTM(256, return_sequences=True))
stateful_model.add(LSTM(128, return_sequences=True, stateful=True))
stateful_model.add(LSTM(32, return_sequences=True, stateful=True))
stateful_model.add(TimeDistributed(Dense(2, activation='softmax')))

stateful_model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
lstm_4 (LSTM)                    (1, 2, 256)           296960      lstm_input_2[0][0]               
____________________________________________________________________________________________________
lstm_5 (LSTM)                    (1, 2, 128)           197120      lstm_4[0][0]                     
____________________________________________________________________________________________________
lstm_6 (LSTM)                    (1, 2, 32)            20608       lstm_5[0][0]                     
____________________________________________________________________________________________________
timedistributed_2 (TimeDistribut (1, 2, 2)             66          lstm_6[0][0]                     
Total params: 514,754
Trainable params: 514,754
Non-trainable params: 0
___________________

In [18]:
adam = optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)

model.compile(optimizer=adam, loss='categorical_crossentropy', metrics=['acc'])
stateful_model.compile(optimizer=adam, loss='categorical_crossentropy', metrics=['acc'])

# model.compile(optimizer=adam, loss='mean_squared_error')

In [40]:
model.fit(x_train, y_train, nb_epoch=300, batch_size = 512)

# adam = optimizers.Adam(lr=0.0001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
# model.compile(optimizer=adam, loss='categorical_crossentropy', metrics=['acc'])
# model.fit(x_train, y_train, nb_epoch=300, batch_size = 512)



# for i in range(x_train.shape[0]):
#   model.fit(x_train[i:(i+1),:,:], y_train[i:(i+1),:,:], nb_epoch=1, batch_size = 1)
#   model.reset_states()

Epoch 1/300
32358/32358 [==============================] - 18s - loss: 0.0036 - acc: 0.9996    
Epoch 2/300
32358/32358 [==============================] - 18s - loss: 0.0036 - acc: 0.9996    
Epoch 3/300
32358/32358 [==============================] - 14s - loss: 0.0036 - acc: 0.9996    
Epoch 4/300
32358/32358 [==============================] - 14s - loss: 0.0036 - acc: 0.9996    
Epoch 5/300
32358/32358 [==============================] - 14s - loss: 0.0036 - acc: 0.9996    
Epoch 6/300
32358/32358 [==============================] - 14s - loss: 0.0036 - acc: 0.9996    
Epoch 7/300
32358/32358 [==============================] - 13s - loss: 0.0035 - acc: 0.9996    
Epoch 8/300
32358/32358 [==============================] - 13s - loss: 0.0035 - acc: 0.9996    
Epoch 9/300
32358/32358 [==============================] - 13s - loss: 0.0036 - acc: 0.9997    
Epoch 10/300
32358/32358 [==============================] - 13s - loss: 0.0036 - acc: 0.9996    
Epoch 11/300
32358/32358 [=============

In [41]:
model.save_weights('/Users/xiaoyudai/Documents/Paper3/Data/code/dnn/model_rnn3')
# model.load_weights('/Users/xiaoyudai/Documents/Paper3/Data/code/dnn/model_rnn_label')

In [49]:
x_test1 = test[:,0:-1].astype(np.float)
sub_length = 4
x_test1 = x_test1.reshape(x_test1.shape[0]/sub_length, sub_length, 33, order='C')

model1 = Sequential()
model1.add(LSTM(256, input_shape=(sub_length, 33), return_sequences=True))
model1.add(LSTM(128, return_sequences=True))
model1.add(LSTM(32, return_sequences=True))
model1.add(TimeDistributed(Dense(2, activation='softmax')))

model1.set_weights(model.get_weights())
pred_prob = model1.predict(x_test1)

In [47]:
pred_prob = model1.predict(x_test)

In [44]:
stateful_model.set_weights(model.get_weights())

pred_prob = stateful_model.predict(x_test, batch_size=1)

In [42]:
pred_prob = model.predict(x_test)

In [22]:
pred_prob[:,:,1].flatten()

array([ 0.2868624 ,  0.96147412,  0.99213749, ...,  0.00378944,
        0.00384939,  0.00388956], dtype=float32)

In [39]:
pred_prob_1 = pred_prob[:,:,1].flatten()
pred = pred_prob_1 > 0.5
truth = test[:,-1].astype(np.float)
confusion_matrix(pred, truth)

array([[288171,  15415],
       [ 10416,   9578]])

In [50]:
pred_prob_1 = pred_prob[:,:,1].flatten()
pred = pred_prob_1 > 0.5
truth = test[:,-1].astype(np.float)
confusion_matrix(pred, truth)

array([[288073,  17442],
       [ 10514,   7551]])

In [ ]:
model.evaluate(x_test, y_test)

In [40]:
pred_prob = pred_prob[:,:,1].flatten()

In [132]:
pred_prob = model.predict(x_test, batch_size=1)
pred_prob = pred_prob[:,:,1].flatten()
pred = pred_prob > 0.5
truth = test[:,-1].astype(np.float)
confusion_matrix(pred, truth)

array([[291043,  17599],
       [  7544,   7394]])

In [102]:
pred_prob

array([  7.72248983e-01,   7.80370474e-01,   9.91780221e-01, ...,
         4.79722105e-04,   4.12322097e-02,   3.80158782e-01], dtype=float32)

In [121]:
np.savetxt('/Users/xiaoyudai/Documents/Paper3/Data/Brain_ES_chr18/crf/result_rnn', pred_prob)

In [79]:
pred.summary()

AttributeError: 'numpy.ndarray' object has no attribute 'summary'